In [1]:
import setpath

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


from prepare.show import get_material_dir
from run.start import get_device

import torch
from prepare.eegdataset import C_GeneralEEGImageDataset, GeneralEEGImageDataset, GeneralEEGPointDataset, N2_GeneralEEGImageDataset

from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np
import os

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from run.others import SmallNet
from prepare.data import filter_two_samples
import run.resnet as resnet
import torch.nn as nn
data_path = '/data0/tianjunchao/dataset/CVPR2021-02785/data/img_pkl/32x32'

from utils.eegutils import getNow

dataset = GeneralEEGPointDataset(path=data_path, n_channels=1, grid_size=8)
# dataset = get_test_setting(dataset)
train_loader = DataLoader(dataset, batch_size=128)
device = get_device()
material_dir = get_material_dir()

[0, 0, 0, 0, 0, 0, 0, 0]
device: cuda:0


In [2]:
# 定义超参数
input_size = 717
hidden_size = 256
num_layers = 2
num_classes = 40
learning_rate = 0.001
num_epochs = 100

In [3]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
# mlp model, 2 layers, relu，dropout, batchnorm
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.base = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.BatchNorm1d(hidden_size)
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.base(x)
        out = self.fc(x)
        return out

In [4]:
# model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)
model = MLPModel(input_size, hidden_size, num_classes).to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    # compute acc
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.squeeze()
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, labels)

        # 统计预测信息
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # 反向传播并优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}%'
                     .format(epoch+1, num_epochs, i+1, total_step, loss.item(), 100 * correct / total))
        
    # test model
    # model.eval()
    # with torch.no_grad():
    #     correct = 0
    #     total = 0
    #     for inputs, labels in train_loader:
    #         inputs = inputs.squeeze()
    #         inputs = inputs.to(device)
    #         labels = labels.to(device)
    #         outputs = model(inputs)
    #         _, predicted = torch.max(outputs.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
    #     print('Test Accuracy of the model on the train images: {} %'.format(100 * correct / total))
            

Epoch [1/100], Step [10/313], Loss: 3.8078, Acc: 2.58%
Epoch [1/100], Step [20/313], Loss: 3.7439, Acc: 2.66%
Epoch [1/100], Step [30/313], Loss: 3.7580, Acc: 2.63%
Epoch [1/100], Step [40/313], Loss: 3.7730, Acc: 2.79%
Epoch [1/100], Step [50/313], Loss: 3.7915, Acc: 2.73%
Epoch [1/100], Step [60/313], Loss: 3.7617, Acc: 2.72%
Epoch [1/100], Step [70/313], Loss: 3.7123, Acc: 2.67%
Epoch [1/100], Step [80/313], Loss: 3.7479, Acc: 2.72%
Epoch [1/100], Step [90/313], Loss: 3.7687, Acc: 2.75%
Epoch [1/100], Step [100/313], Loss: 3.7252, Acc: 2.74%
Epoch [1/100], Step [110/313], Loss: 3.7279, Acc: 2.79%
Epoch [1/100], Step [120/313], Loss: 3.7337, Acc: 2.81%
Epoch [1/100], Step [130/313], Loss: 3.7153, Acc: 2.76%
Epoch [1/100], Step [140/313], Loss: 3.7247, Acc: 2.76%
Epoch [1/100], Step [150/313], Loss: 3.7571, Acc: 2.76%
Epoch [1/100], Step [160/313], Loss: 3.7144, Acc: 2.71%
Epoch [1/100], Step [170/313], Loss: 3.7308, Acc: 2.71%
Epoch [1/100], Step [180/313], Loss: 3.7469, Acc: 2.70%
E

KeyboardInterrupt: 

In [ ]:
outputs.shape

In [ ]:
inputs.shape